##### Copyright 2022 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# assignment_mb

<table align="left">
<td>
<a href="https://colab.research.google.com/github/google/or-tools/blob/main/examples/notebook/linear_solver/assignment_mb.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/colab_32px.png"/>Run in Google Colab</a>
</td>
<td>
<a href="https://github.com/google/or-tools/blob/main/ortools/linear_solver/samples/assignment_mb.py"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/github_32px.png"/>View source on GitHub</a>
</td>
</table>

First, you must install [ortools](https://pypi.org/project/ortools/) package in this colab.

In [ ]:
!pip install ortools

MIP example that solves an assignment problem.

In [ ]:
from ortools.linear_solver.python import model_builder


def main():
    # Data
    costs = [
        [90, 80, 75, 70],
        [35, 85, 55, 65],
        [125, 95, 90, 95],
        [45, 110, 95, 115],
        [50, 100, 90, 100],
    ]
    num_workers = len(costs)
    num_tasks = len(costs[0])

    # Solver
    # Create the model.
    model = model_builder.ModelBuilder()

    # Variables
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if worker i is assigned to task j.
    x = {}
    for i in range(num_workers):
        for j in range(num_tasks):
            x[i, j] = model.new_bool_var(f'x_{i}_{j}')

    # Constraints
    # Each worker is assigned to at most 1 task.
    for i in range(num_workers):
        model.add(sum(x[i, j] for j in range(num_tasks)) <= 1)

    # Each task is assigned to exactly one worker.
    for j in range(num_tasks):
        model.add(sum(x[i, j] for i in range(num_workers)) == 1)

    # Objective
    objective_expr = 0
    for i in range(num_workers):
        for j in range(num_tasks):
            objective_expr += costs[i][j] * x[i, j]
    model.minimize(objective_expr)

    # Create the solver with the CP-SAT backend, and solve the model.
    solver = model_builder.ModelSolver('sat')
    status = solver.solve(model)

    # Print solution.
    if (status == model_builder.SolveStatus.OPTIMAL or
            status == model_builder.SolveStatus.FEASIBLE):
        print(f'Total cost = {solver.objective_value}\n')
        for i in range(num_workers):
            for j in range(num_tasks):
                # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                if solver.value(x[i, j]) > 0.5:
                    print(f'Worker {i} assigned to task {j}.' +
                          f' Cost: {costs[i][j]}')
    else:
        print('No solution found.')


main()

